# Data cleaning

In [1]:
import pandas as pd
import numpy as np
import glob

## 1 - Data acquisition

### 1.1 - Data cleaning pipeline configuration

In [2]:
def adolescent_mom(x):
    if x<18: return 1
    if x>18: return 0
    else: return np.nan
def premature(x):
    if x in [1, 2, 3, 4]: return 1
    if x in [5, 6]: return 0
    else: return np.nan

In [3]:
columns = [
    "GESTACAO", "LOCNASC",
    "IDADEMAE", "ESTCIVMAE", "ESCMAE", "QTDFILVIVO",
    "QTDFILMORT", "CODMUNRES", "GRAVIDEZ", "PARTO",
    "CONSULTAS", "SEXO",
    "IDANOMAL",
    "RACACORMAE", "IDADEPAI", "SEMAGESTAC", "TPAPRESENT",
    "STTRABPART"
]

In [8]:
def clean_data(raw_data):
    cleaned_data = (
        raw_data
        .assign(
            premature = lambda df: (
                df["GESTACAO"].apply(premature)
            ),
            adolescent_mom = lambda df: (
                df["IDADEMAE"].apply(adolescent_mom)
            ),
            place_of_birth = lambda df: (
                df["LOCNASC"]
                .replace({
                    9:np.nan,
                    1:"hospital", 
                    2:"another_health_est",
                    3:"home",
                    4:"others"
                })
            ),
            civil_sate_mom = lambda df: (
                df["ESTCIVMAE"]
                .replace({
                    1:"single", 
                    2:"married",
                    3:"widow",
                    4:"divorced",
                    5:"union",
                    9:np.nan 
                })
            ),
            mom_schooling = lambda df: (
                df["ESCMAE"]
                .replace({ 
                    1:"none",
                    2:"1_to_3_years",
                    3:"4_to_7_years",
                    4:"8_to_11_years",
                    5:"12_more_years",
                    9:np.nan
                })
            ),
            pregnancy = lambda df: (
                df["GRAVIDEZ"]
                .replace({
                    1:"unique", 
                    2:"pair",
                    3:"triple_more",
                    9:np.nan                    
                })
            ),
            childbirth = lambda df: (
                df["PARTO"]
                .replace({
                    1:"vaginal",
                    2:"cesarean",
                    9:np.nan
                })
            ),
            prenatal_appointments = lambda df: (
                df["CONSULTAS"]
                .replace({
                    1:"0",
                    2:"1_to_3",
                    3:"4_to_6",
                    4:"7_more",
                    9:np.nan                    
                })
            ),
            sex = lambda df: (
                df["SEXO"]
                .replace({
                    0:np.nan,
                    1:"masculine",
                    2:"feminine"
                })
            ),
            anomaly = lambda df: (
                df["IDANOMAL"]
                .replace({
                    9:np.nan,
                    1:"yes",
                    2:"no"
                })
            ),
            mom_school_level = lambda df: (
                df["ESCMAE"]
                .replace({
                    0:"none",
                    1:"fundamental_i",
                    2:"fundamental_ii",
                    3:"medium",
                    4:"unfinished_superior",
                    5:"finished_superior",
                    9:np.nan
                })
            ),
            mom_race = lambda df: (
                df["RACACORMAE"]
                .replace({
                    1:"white",
                    2:"black",
                    3:"yellow",
                    4:"brown",
                    5:"indigenous"
                })
            ),
            baby_position = lambda df: (
                df["TPAPRESENT"]
                .replace({
                    1:"cephalic",
                    2:"pelvic_or_podalic",
                    3:"transverse",
                    9:np.nan                    
                })
            ),
            induced_labor = lambda df: (
                df["STTRABPART"]
                .replace({
                    1:"yes",
                    2:"no",
                    3:"do_not_apply",
                    9:np.nan                    
                })
            ),
            dad_age = lambda df: (
                pd.to_numeric(df["IDADEPAI"], errors="coerce")
            )
        )
        .drop(columns=[
            "GESTACAO", "LOCNASC", "ESTCIVMAE", "ESCMAE", 
            "GRAVIDEZ", "PARTO", "CONSULTAS", "SEXO",
            "IDANOMAL", "ESCMAE", "RACACORMAE", "TPAPRESENT",
            "STTRABPART", "IDADEPAI"
        ])
        .rename(columns={
            "QTDFILVIVO":"number_alive_children",
            "QTDFILMORT":"number_dead_children",
            "IDADEMAE":"mom_age",
            "SEMAGESTAC":"gestational_week", 
        })
        .query("premature == 0 | premature == 1")
        .reset_index(drop=True)
    )
    
    return cleaned_data

In [5]:
def merge_geoses(data, geoses):
    merged_data = (
        data
        .merge(
            geoses[["MUNIC_CODE6", "GeoSES", "GeoSESed", "GeoSESpv", "GeoSESdp", "GeoSESwl", "GeoSESin", "GeoSESsg"]],
            left_on="CODMUNRES",
            right_on="MUNIC_CODE6",
            how="left"
        )
        .drop(columns=["MUNIC_CODE6", "CODMUNRES"])
        .rename(columns={
            "GeoSES":"geoses",
            "GeoSESed":"geoses_education",
            "GeoSESpv":"geoses_poverty",
            "GeoSESdp":"geoses_deprivation",
            "GeoSESwl":"geoses_wealth",
            "GeoSESin":"geoses_income",
            "GeoSESsg":"geoses_segregation"
        })
    )
    return merged_data

### 1.2 - GeoSES

In [12]:
df_geoses = pd.read_excel("./data/geoses/pone.0232074.s003.xls", sheet_name="BR_variables")

### 1.3 - SP - 2010 - 2020

In [6]:
year_list = np.arange(2010, 2021)
filelist = []
for year in year_list:
    filelist += glob.glob(f"./data/sinasc/*SP_{year}_t.csv")
df_sinasc_sp_raw = pd.concat([pd.read_csv(file, usecols=columns) for file in filelist]).reset_index(drop=True)

C:\Users\drt61206\AppData\Local\Temp/ipykernel_13832/4253760115.py:5: DtypeWarning: Columns (44) have mixed types.Specify dtype option on import or set low_memory=False.
  df_sinasc_sp_raw = pd.concat([pd.read_csv(file, usecols=columns) for file in filelist]).reset_index(drop=True)


In [9]:
df_sinasc_sp_clean = clean_data(df_sinasc_sp_raw)

In [10]:
del(df_sinasc_sp_raw)

In [13]:
df_sinasc_sp_geoses = merge_geoses(df_sinasc_sp_clean, df_geoses)

In [16]:
df_sinasc_sp_geoses[df_sinasc_sp_geoses.select_dtypes("object").columns.tolist()] = df_sinasc_sp_geoses[df_sinasc_sp_geoses.select_dtypes("object").columns.tolist()].astype(str)

In [19]:
df_sinasc_sp_geoses.to_parquet("./data/sinasc_sp_10_a_20.parquet", index=False)

In [20]:
del(df_sinasc_sp_geoses, df_sinasc_sp_clean)

### 1.2 - Brazil - 2019, 2020

In [22]:
df_sinasc_br_19_20_raw = (
    pd.concat([
        pd.read_csv(file, usecols=columns)
        for file
        in glob.glob("./data/sinasc/*2019_t.csv") + glob.glob("./data/sinasc/*2020_t.csv")
    ])
    .reset_index(drop=True)
)

In [23]:
df_sinasc_br_19_20_clean = clean_data(df_sinasc_br_19_20_raw)

In [24]:
del(df_sinasc_br_19_20_raw)

In [25]:
df_sinasc_br_geoses = merge_geoses(df_sinasc_br_19_20_clean, df_geoses)

In [26]:
df_sinasc_br_geoses[df_sinasc_br_geoses.select_dtypes("object").columns.tolist()] = df_sinasc_br_geoses[df_sinasc_br_geoses.select_dtypes("object").columns.tolist()].astype(str)

In [27]:
df_sinasc_br_geoses.to_parquet("./data/sinasc_br_19_20.parquet", index=False)